In [1]:
import re
import os
import sys
import cv2
import math
import time
import shutil

import pandas as pd
import numpy as np
import pydicom
from utils import *

import matplotlib.pyplot as plt
from tqdm import tqdm
from shutil import copy
from datetime import date
from pydicom.uid import UID, generate_uid
from multiprocessing import dummy as multiprocessing

# ROI dcm to png frame
- process
    - dcm header 정보를 이용하여 crop
    - cv2 method를 이용하여 한번더 crop

- version_1에서 동일한 영역이 없어서 제외한 case
    - study_id : 61398238_4, 61398238_7

In [53]:
# 1차 수집
v1_video_df = pd.read_csv('/home/work/LUS/Dataset/csv_files/version1_v1.csv', index_col=False)
v2_video_df = pd.read_csv('/home/work/LUS/Dataset/csv_files/version2_v1.csv', index_col=False)

In [54]:
print(len(v1_video_df))
print(len(v2_video_df))

v1_video_df.head()

388
388


,Unnamed: 0,PatientID,Data_idx,StudyDate,Data_type,A-line,B-line,Confluent B-line,Consolidation,Pleural effusion,Lung sliding,Lung Ultrasound Score,study_id
0,0,24965778,2,20230614,video,NaN,24_32,NaN,NaN,NaN,NaN,1,24965778_2
1,1,24965778,3,20230614,video,NaN,24_27,NaN,NaN,NaN,NaN,1,24965778_3
2,2,24965778,4,20230614,video,NaN,NaN,NaN,2_18,NaN,NaN,3_c,24965778_4
3,3,24965778,5,20230614,video,NaN,NaN,NaN,NaN,1_54,NaN,3_e,24965778_5
4,4,24965778,6,20230614,video,15_60,NaN,NaN,NaN,NaN,NaN,0,24965778_6


In [55]:
def create_raw_dcm_path(row):
    pid = row['PatientID']
    data_idx = row['Data_idx']
    return f"/home/work/LUS/Dataset/processed_dataset_dcm/{pid}/{str(data_idx).zfill(5)}.dcm"
def create_avi_dcm_path(row):
    pid = row['PatientID']
    data_idx = row['Data_idx']
    return f"/home/work/LUS/Dataset/processed_dataset_avi/{pid}/{str(data_idx).zfill(5)}.avi"

v1_video_df['dcm_path'] = v1_video_df.apply(create_raw_dcm_path, axis=1)
v1_video_df['avi_path'] = v1_video_df.apply(create_avi_dcm_path, axis=1)
v2_video_df['dcm_path'] = v2_video_df.apply(create_raw_dcm_path, axis=1)
v2_video_df['avi_path'] = v2_video_df.apply(create_avi_dcm_path, axis=1)

In [56]:
v1_video_df[v1_video_df['PatientID'] == 60241834]

,Unnamed: 0,PatientID,Data_idx,StudyDate,Data_type,A-line,B-line,Confluent B-line,Consolidation,Pleural effusion,Lung sliding,Lung Ultrasound Score,study_id,dcm_path,avi_path
282,302,60241834,2,20230906,video,NaN,1_152,NaN,NaN,NaN,NaN,1,60241834_2,/home/work/LUS/Dataset/processed_dataset_dcm/6...,/home/work/LUS/Dataset/processed_dataset_avi/6...
283,303,60241834,3,20230906,video,NaN,1_152,NaN,NaN,NaN,NaN,1,60241834_3,/home/work/LUS/Dataset/processed_dataset_dcm/6...,/home/work/LUS/Dataset/processed_dataset_avi/6...
284,304,60241834,4,20230906,video,NaN,1_152,NaN,NaN,NaN,NaN,1,60241834_4,/home/work/LUS/Dataset/processed_dataset_dcm/6...,/home/work/LUS/Dataset/processed_dataset_avi/6...
285,306,60241834,6,20230906,video,NaN,1_152,NaN,NaN,NaN,NaN,1,60241834_6,/home/work/LUS/Dataset/processed_dataset_dcm/6...,/home/work/LUS/Dataset/processed_dataset_avi/6...
286,307,60241834,7,20230906,video,NaN,NaN,NaN,1_152,NaN,NaN,3_c,60241834_7,/home/work/LUS/Dataset/processed_dataset_dcm/6...,/home/work/LUS/Dataset/processed_dataset_avi/6...
287,308,60241834,8,20230906,video,NaN,1_82,NaN,NaN,NaN,NaN,3_c,60241834_8,/home/work/LUS/Dataset/processed_dataset_dcm/6...,/home/work/LUS/Dataset/processed_dataset_avi/6...
288,309,60241834,9,20230906,video,NaN,NaN,NaN,13_43,NaN,NaN,3_c,60241834_9,/home/work/LUS/Dataset/processed_dataset_dcm/6...,/home/work/LUS/Dataset/processed_dataset_avi/6...
289,310,60241834,10,20230906,video,NaN,NaN,NaN,1_152,NaN,NaN,3_c,60241834_10,/home/work/LUS/Dataset/processed_dataset_dcm/6...,/home/work/LUS/Dataset/processed_dataset_avi/6...
290,311,60241834,11,20230906,video,NaN,NaN,NaN,1_152,NaN,NaN,3_c,60241834_11,/home/work/LUS/Dataset/processed_dataset_dcm/6...,/home/work/LUS/Dataset/processed_dataset_avi/6...
291,312,60241834,12,20230906,video,NaN,NaN,NaN,1_152,NaN,NaN,3_c,60241834_12,/home/work/LUS/Dataset/processed_dataset_dcm/6...,/home/work/LUS/Dataset/processed_dataset_avi/6...


In [57]:
def dcm_frames_to_png(video_path, destinationFolder, folder_name, file_name, convert_type):
    # fileName = dicom_path.split('/')[-1]

    dataset = pydicom.dcmread(video_path, force=True)
    if dataset.PixelData:
        test_arr = dataset.pixel_array
        
        # 동영상인 case
        if len(test_arr.shape) == 4:  

            # get information to crop each frame
            xmin = dataset[(0x018, 0x6011)].value[0][(0x018, 0x6018)].value
            ymin = dataset[(0x018, 0x6011)].value[0][(0x018, 0x601a)].value
            xmax = dataset[(0x018, 0x6011)].value[0][(0x018, 0x601c)].value
            ymax = dataset[(0x018, 0x6011)].value[0][(0x018, 0x601e)].value

            frames, height, width, channels = test_arr.shape
            cropped_height = ymax - ymin
            cropped_width = xmax - xmin

            try:
                fps = dataset[(0x18, 0x40)].value
            except:
                print("couldn't find frame rate, default to 30")
                fps = 30

            if convert_type == 'ALL':
                for i in range(frames):
                    output = test_arr[i, :, :, 0]

                    # crop image
                    cropped_frame = output[ymin:ymax, xmin:xmax]

                    # Convert to 3-channel image
                    final_output = cv2.merge([cropped_frame, cropped_frame, cropped_frame])

                    # Save each frame as PNG
                    png_filename = os.path.join(destinationFolder, folder_name, f"{folder_name}_{file_name.replace('.dcm', '')}_{i+1}.png")
                    cv2.imwrite(png_filename, final_output)

#                 print(f'Frames saved as PNG files in {png_filename}')
                
#             elif convert_type == 'ROI':    
                
#                 for i in range(start_frame-1, end_frame):
#                     output = test_arr[i, :, :, 0]

#                     # crop image
#                     cropped_frame = output[ymin:ymax, xmin:xmax]

#                     # Convert to 3-channel image
#                     final_output = cv2.merge([cropped_frame, cropped_frame, cropped_frame])

#                     # Save each frame as PNG
#                     png_filename = os.path.join(destinationFolder, folder_name, f"{folder_name}_{file_name.replace('.dcm', '')}_{i+1}.png")
#                     cv2.imwrite(png_filename, final_output)

#                 print(f'Frames saved as PNG files in {destinationFolder}')
        
        # image case
        elif len(test_arr.shape) == 3:
            
            # get ionformation to crop each frame
            xmin = dataset[(0x018, 0x6011)].value[0][(0x018, 0x6018)].value
            ymin = dataset[(0x018, 0x6011)].value[0][(0x018, 0x601a)].value
            xmax = dataset[(0x018, 0x6011)].value[0][(0x018, 0x601c)].value
            ymax = dataset[(0x018, 0x6011)].value[0][(0x018, 0x601e)].value
            
            height, width, channels = test_arr.shape
            cropped_height = ymax - ymin
            cropped_width = xmax - xmin
            
            # crop image
            cropped_frame = test_arr[ymin:ymax, xmin:xmax]

            # Convert to 3-channel image
#             final_output = cv2.merge([cropped_frame, cropped_frame, cropped_frame])

            # Save each frame as PNG
            png_filename = os.path.join(destinationFolder, folder_name, f"{folder_name}_{file_name.replace('.dcm', '')}.png")
            cv2.imwrite(png_filename, cropped_frame)
            print(f'Frames saved as PNG files in {destinationFolder}')
            
    else:
        print(f'{file_name} is empty.')
        return 0

In [58]:
def dcm_frames_to_png_v2(video_path, destinationFolder, folder_name, file_name, convert_type):
    # fileName = dicom_path.split('/')[-1]

    dataset = pydicom.dcmread(video_path, force=True)
    if dataset.PixelData:
        test_arr = dataset.pixel_array
        
        # 동영상인 case
        if len(test_arr.shape) == 4:

            frames, height, width, channels = test_arr.shape

            try:
                fps = dataset[(0x18, 0x40)].value
            except:
                print("couldn't find frame rate, default to 30")
                fps = 30

            if convert_type == 'ALL':
                for i in range(frames):
                    org_img = test_arr[i, :, :, 0]
                    
                    # snuh 기준
                    x, y, w, h = 160, 109, 989, 729
                    cropped_img = org_img[y:y+h, x:x+w]
                    
                    cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_GRAY2BGR)
                    
                    # Save each frame as PNG
                    png_filename = os.path.join(destinationFolder, folder_name, f"{folder_name}_{file_name.replace('.dcm', '')}_{i+1}.png")
                    cv2.imwrite(png_filename, cropped_img)
        
        # image case
        else:
#             print(f'{file_name} is not a video file.')
            return 0
#         elif len(test_arr.shape) == 3:
            
#             # snuh 기준
#             x, y, w, h = 160, 109, 989, 729
#             cropped_img = test_arr[y:y+h, x:x+w]

#             png_filename = os.path.join(destinationFolder, folder_name, f"{folder_name}_{file_name.replace('.dcm', '')}.png")
#             cv2.imwrite(png_filename, cropped_img)
#             print(f'Frames saved as PNG files in {destinationFolder}')
            
    else:
        print(f'{file_name} is empty.')
        return 0

## version 1

In [23]:
pwd

'/home/work/LUS/code'

### all dcm to png frame

In [21]:
processed_dcm_head_path = '/home/work/LUS/Dataset/processed_dataset_dcm'
destination_folder = '/home/work/LUS/Dataset/dcm_to_png/all_frame/version_1'

subfolders = os.listdir(processed_dcm_head_path)

count = 0

for idx, row in  tqdm(v1_video_df.iterrows(), total=len(v1_video_df)):
    path = row.dcm_path
    folder_name = path.split('/')[-2]
    file_name = path.split('/')[-1]
        
    count += 1
    
    if not os.path.exists(os.path.join(destination_folder, folder_name, file_name)):
        os.makedirs(os.path.join(destination_folder, folder_name), exist_ok=True)
    
    # convert dcm to png
    dcm_frames_to_png(path, destination_folder, folder_name, file_name, convert_type= 'ALL')

100% 54/54 [05:31<00:00,  6.13s/it]


In [60]:
processed_dcm_head_path = '/home/work/LUS/Dataset/processed_dataset_dcm'
destination_folder = '/home/work/LUS/Dataset/dcm_to_png/all_frame/version_2'

subfolders = os.listdir(processed_dcm_head_path)

count = 0

for idx, row in  tqdm(v2_video_df.iterrows(), total=len(v2_video_df)):
    path = row.dcm_path
    folder_name = path.split('/')[-2]
    file_name = path.split('/')[-1]
        
    count += 1
    
    if not os.path.exists(os.path.join(destination_folder, folder_name, file_name)):
        os.makedirs(os.path.join(destination_folder, folder_name), exist_ok=True)
    
    # convert dcm to png
    dcm_frames_to_png(path, destination_folder, folder_name, file_name, convert_type= 'ALL')

100% 349/349 [32:43<00:00,  5.63s/it]


In [35]:
destination_folder = '/home/work/LUS/Dataset/dcm_to_png/all_frame/version_1'

In [36]:
folder_name = '60241834'
file_name = '00005'
dcm_frames_to_png(a, destination_folder, folder_name, file_name, convert_type= 'ALL')
